In [7]:
#!pip install opencv-python

import numpy as np
import cv2

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 1.9 MB/s eta 0:00:21
   ---------------------------------------- 0.2/38.8 MB 2.9 MB/s eta 0:00:14
   ---------------------------------------- 0.4/38.8 MB 3.2 MB/s eta 0:00:13
    --------------------------------------- 0.5/38.8 MB 3.4 MB/s eta 0:00:12
    --------------------------------------- 0.7/38.8 MB 3.3 MB/s eta 0:00:12
    --------------------------------------- 0.8/38.8 MB 3.2 MB/s eta 0:00:13
   - -------------------------------------- 1.0/38.8 MB 3.3 MB/s eta 0:00:12
   - -------------------------------------- 1.2/38.8 MB 3.4 MB/s eta 0:00:12
   - -------------------------------------- 1.3/38.8 MB 3.4 MB/s eta 0:00:12
   - ----------

In [198]:
# Paths to model files and input image
prototxt_path = "Models/colorization_deploy_v2.prototxt"

# The file "colorization_release_v2.caffemodel"  is not on my git cause the model was to large
# you can download it on this link : https://drive.google.com/drive/folders/1FaDajjtAsntF_Sw5gqF0WyakviA5l8-a
model_path = "Models/colorization_release_v2.caffemodel"

kernel_path = "Models/pts_in_hull.npy"
image_path = "Images/tiger_B&W.jpg"

In [199]:

# Load the pre-trained colorization model
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Load the cluster centers
points = np.load(kernel_path)

In [200]:

# Reshape the points to match the network's input requirements
points = points.transpose().reshape(2, 313 , 1, 1 )

# Set the cluster centers as the weights of the network's layers
net.getLayer(net.getLayerId("class8_ab")).blobs = [points.astype(np.float32)]
net.getLayer(net.getLayerId("conv8_313_rh")).blobs = [np.full([1,313], 2.606, dtype="float32")]

# LAB -> L = Lightness a* b*


# Read and normalize the black and white image
bw_image = cv2.imread(image_path)
normalized = bw_image.astype("float32") / 255.0

# Convert the image to the LAB color space
lab = cv2.cvtColor(normalized, cv2.COLOR_BGR2LAB)


In [201]:

# Resize the image to match the input size of the network
resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]

# Subtract 50 to center the Lightness values / can be adjusted
L -= 50

In [202]:

# Pass the L channel through the network to predict the ab channels
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1,2,0))

# Resize the ab channels to match the original image size
ab = cv2.resize(ab, (bw_image.shape[1], bw_image.shape[0]))
L= cv2.split(lab)[0]

In [203]:
# Combine the L and ab channels and convert back to the BGR color space
colorized = np.concatenate((L[:,:,np.newaxis], ab), axis=2)
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = (255.0 * colorized).astype("uint8")

In [204]:
# Display the original and colorized images
cv2.imshow("BW Image", bw_image)
cv2.imshow("Colorized", colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [197]:
#To download the image colorized
#cv2.imwrite('Images/colorized_image.jpg', colorized)

True